<a href="https://colab.research.google.com/github/malcolmfisher103/Bioinformatic-Scripts/blob/main/Parsing_RNA_SEQ_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Set GSE
GSE_ID = "GSE41338" # @param {type:"string"}
Genome_build = "XENTR_10.0" # @param ["XENLA_10.1", "XENTR_10.0"]

In [ ]:
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np  # Add this line
from matplotlib.ticker import LogLocator
from scipy.interpolate import interp1d

In [ ]:
def load_tpm_matrix():
    return pd.read_csv(f'https://bigfrog.xenbase.org/xenbase/genomics/GEO/{GSE_ID}/{Genome_build}/RNA-Seq/ExpressionFiles/Genes_TPM_Matrix.txt', sep='\t', index_col=0)

def load_gene_info():
    return pd.read_csv('https://xenbase-bio1.ucalgary.ca/cgi-bin/reports/models_gene_info.cgi', sep='\t', index_col=0)

def load_gsm_sample_mapping():
    return pd.read_csv(f'https://bigfrog.xenbase.org/xenbase/genomics/GEO/{GSE_ID}/{Genome_build}/RNA-Seq/gsm_to_track.txt', sep='\t')

def load_gsm_srr_mapping():
    return pd.read_csv('https://download.xenbase.org/xenbase/GenePageReports/geo_srr_metadata_chd.txt', sep='\t')

def load_gse_species_mapping():
    return pd.read_csv('https://download.xenbase.org/xenbase/GenePageReports/geo_metadat_chd.txt', sep='\t')

def create_gsm_to_srr_mapping(gsm_srr_mapping):
    return gsm_srr_mapping[['SRR', 'GSM']]

In [ ]:
def substitute_gene_symbols(tpm_matrix, gene_info):

# Drop rows with null values in column 1 and column 2 in file 2
  gene_info = gene_info.dropna(subset=['GENE_SYMBOL', 'MODEL_NAME'])

# Drop duplicates in file 2 based on column 2
  gene_info = gene_info.drop_duplicates(subset=['MODEL_NAME'])

# Create a mapping dictionary from column 2 to column 1 in file 2
  mapping = dict(zip(gene_info['MODEL_NAME'], gene_info['GENE_SYMBOL']))

# Replace values in column 1 of file 1 with corresponding values from file 2
  tpm_matrix['Gene'] = tpm_matrix['Gene'].map(mapping).fillna(tpm_matrix['Gene'])

# Save the modified DataFrame back to file1.csv or use it as needed
#tpm_matrix.to_csv('modified_file1.csv', index=False)
  return tpm_matrix

In [ ]:
def process_data(tpm_matrix, gene_info, gsm_sample_mapping, gsm_srr_mapping):
    # Substituting gene symbols
    tpm_matrix = substitute_gene_symbols(tpm_matrix, gene_info) #This works

    # Mapping GSMs to SRRs
    gsm_to_srr = create_gsm_to_srr_mapping(gsm_srr_mapping)

    # Select columns 2 and 6 to create a DataFrame with Track Name and GSMs
    gsm_mapping = gsm_sample_mapping[['Track Name', 'GSMs']]

    # Convert GSMs column to list if it contains comma-separated values
    #gsm_mapping['GSMs'] = gsm_mapping['GSMs'].str.split(',')
    gsm_mapping.loc[:, 'GSMs'] = gsm_mapping['GSMs'].str.split(',')

    # Explode the list of GSMs to create multiple rows for each track name
    gsm_mapping = gsm_mapping.explode('GSMs')
    # Merge gsm_mapping with gsm_srr_mapping on the 'GSM' column to get corresponding SRRs
    track_srr_mapping = pd.merge(left=gsm_mapping, right=gsm_srr_mapping, left_on='GSMs', right_on='GSM', validate="1:m")
    # Group by 'Track Name' and aggregate the corresponding SRRs into lists
    track_srr_mapping = track_srr_mapping.groupby('Track Name')['SRR'].apply(list).reset_index()
    track_srr_mapping_expanded = track_srr_mapping.explode('SRR')
    #print(track_srr_mapping_expanded.head(10))

    srr_columns = tpm_matrix.columns.intersection(track_srr_mapping_expanded['SRR'])
    #print(srr_columns)
    srr_to_track = dict(zip(track_srr_mapping_expanded['SRR'], track_srr_mapping_expanded['Track Name']))

    # Grouping TPM matrix by SRRs
    tpm_matrix.rename(columns={col: srr_to_track.get(col, col) for col in srr_columns}, inplace=True)
    #print(tpm_matrix.head(10))

    # Melt the DataFrame to have a single column for the gene and the rest for values
    tpm_matrix = pd.melt(tpm_matrix, id_vars=['Gene'], var_name='Column')

    # Extract the unique column names excluding the first column (Gene)
    columns_to_merge = tpm_matrix['Column'].unique()[1:]

    # Group by track names and take the mean
    tpm_matrix_grouped = tpm_matrix.groupby(['Gene', 'Column']).mean().unstack()

    # Display the first 10 rows and first 5 columns
    print(tpm_matrix_grouped.iloc[:10, :5])

    # Remove the hierarchical index and reset index
    tpm_matrix_grouped.columns = tpm_matrix_grouped.columns.droplevel()
    tpm_matrix_grouped.reset_index(inplace=True)

    #print(tpm_matrix_grouped.head(10))

    return tpm_matrix_grouped

In [ ]:
def main():

    # Load data
    tpm_matrix = load_tpm_matrix()
    tpm_matrix = tpm_matrix.reset_index()
    gsm_srr_mapping = load_gsm_srr_mapping()
    gene_info = load_gene_info()
    gene_info = gene_info.reset_index()
    gsm_sample_mapping = load_gsm_sample_mapping()

    # Process data
    new_tpm_matrix = process_data(tpm_matrix, gene_info, gsm_sample_mapping, gsm_srr_mapping)

    # Output new_tpm_matrix to file
    new_tpm_matrix.to_csv('output.txt', sep='\t')

if __name__ == "__main__":
    main()

                 value                                             \
Column   brain - adult heart - adult kidney - adult liver - adult   
Gene                                                                
42sp43            0.00          0.00          22.69          0.00   
42sp50            1.27          0.14          20.20          0.00   
64phr             8.50          0.50           5.60          0.71   
ATP6           6466.53       5914.10        2894.22       2916.53   
ATP8           5943.21      15295.18        4310.74       4103.86   
COX1          24127.09      36673.37       14140.14       8944.64   
COX2          14559.12      36524.54       12763.37      15440.92   
COX3          19206.86      40314.86       12412.92      13062.72   
CYTB           7462.84      14861.47        4463.55       3572.17   
KIAA0040          0.51         21.39           6.85          8.57   

                                  
Column   skeletal muscle - adult  
Gene                            

In [ ]:
!cat output.txt|cut -f2-|grep -a 'Gene\|wnt'

Gene	brain - adult	heart - adult	kidney - adult	liver - adult	skeletal muscle - adult
wnt1	0.58	0.0	0.0	0.0	0.0
wnt10a	0.45	0.0	1.84	0.0	0.27
wnt10b	1.81	0.05	0.17	0.0	0.21
wnt11	3.84	1.89	2.69	3.32	1.14
wnt11b	0.14	0.0	7.07	0.0	0.0
wnt16	0.0	0.0	0.0	0.0	0.0
wnt2	1.23	0.0	0.0	0.0	0.0
wnt2b	1.75	4.29	3.41	0.81	0.08
wnt3	7.03	0.0	0.0	0.0	0.0
wnt3a	0.64	0.0	0.01	0.0	0.0
wnt4	2.56	1.11	8.59	0.04	1.27
wnt5a	4.82	1.01	5.3	0.91	0.36
wnt5b	8.04	8.07	22.15	1.44	1.44
wnt6	7.76	0.0	0.68	0.0	0.0
wnt7a	12.22	0.11	1.12	0.23	0.0
wnt7b	17.05	0.05	10.92	0.0	0.0
wnt7c	4.91	0.03	0.36	0.01	0.03
wnt8a	1.2	0.0	0.0	0.0	0.0
wnt8b	3.68	0.0	0.0	0.0	0.0
wnt9a	0.25	4.52	4.85	6.6	0.47
wnt9b	0.0	0.0	0.37	0.0	0.0


In [ ]:
# Load DataFrame
new_tpm_matrix = pd.read_csv('output.txt', sep='\t', index_col=0)

df = pd.DataFrame(new_tpm_matrix)
df = df[['Gene', 'oocyte - oocyte I-II', 'oocyte - oocyte III-IV', 'oocyte - oocyte V-VI', 'egg - unfertilized egg', 'WE - NF8', 'WE - NF9', 'WE - NF10-10.5', 'WE - NF12', 'WE - NF15', 'WE - NF20', 'WE - NF25', 'WE - NF29/30', 'WE - NF35/36', 'WE - NF40']]

# List of genes to include in the plot
genes_to_plot = ["pax2.L", "nadsyn1.L", "slc7a5.L"]

# Filter the DataFrame to include only the specified genes
filtered_df = df[df['Gene'].isin(genes_to_plot)]

# Plotting
plt.figure(figsize=(10, 6))

for index, row in filtered_df.iterrows():
    gene = row['Gene']
    expression_values = row.drop('Gene').values
    time_points = row.drop('Gene').index

# Perform cubic spline interpolation
    f = interp1d(range(len(time_points)), expression_values, kind='cubic')
    x_new = np.linspace(0, len(time_points) - 1, 100)  # Adjust 100 for desired smoothness

    y_new = np.maximum(f(x_new), 0)

    plt.plot(time_points, expression_values, 'o', label=gene)  # Plot original data points
    plt.plot(x_new, y_new, label=f'{gene} (smoothed)')

plt.xlabel('Time Points')
plt.ylabel('Gene Expression Level')
plt.title('Gene Expression Data')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(True)
#plt.yscale('log')  # Set y-axis to logarithmic scale
#plt.gca().set_yscale('function', functions=(lambda x: np.log2(x), lambda x: np.exp2(x))) # Set y-axis to logarithmic scale with base 2
#plt.gca().yaxis.set_major_locator(LogLocator(base=2.0)) # Use LogLocator for y-axis ticks
plt.tight_layout()
plt.show()

KeyError: "['oocyte - oocyte I-II', 'oocyte - oocyte III-IV', 'oocyte - oocyte V-VI', 'egg - unfertilized egg', 'WE - NF8', 'WE - NF10-10.5', 'WE - NF12', 'WE - NF15', 'WE - NF20', 'WE - NF25', 'WE - NF29/30', 'WE - NF35/36', 'WE - NF40'] not in index"

In [ ]:
# Load DataFrame
data = pd.read_csv('output.txt', sep='\t', index_col=0)

df = pd.DataFrame(data)
df = df[['Gene', 'oocyte - oocyte I-II', 'oocyte - oocyte III-IV', 'oocyte - oocyte V-VI', 'egg - unfertilized egg', 'WE - NF8', 'WE - NF9', 'WE - NF10-10.5', 'WE - NF12', 'WE - NF15', 'WE - NF20', 'WE - NF25', 'WE - NF29/30', 'WE - NF35/36', 'WE - NF40']]


# Remove the 'Gene' column for clustering
gene_names = df['Gene']  # Store gene names for later use
df.drop(columns=['Gene'], inplace=True)

# Define filtering criteria
min_expression = 5.0  # Minimum expression level
min_timepoints_above_threshold = 3  #

# Drop non-varying genes (standard deviation = 0)
df_filtered = df[(df > min_expression).sum(axis=1) >= min_timepoints_above_threshold]
df_filtered = df_filtered[df_filtered.std(axis=1) > 2]

print(df_filtered.head(10))

# Apply log2 transform to the expression values
df_filtered_log2 = np.log2(df_filtered)

# Generate a clustered heatmap
plt.figure(figsize=(10, 6))
# sns.heatmap((df_filtered.head(100)), cmap='viridis', annot=False, fmt=".2f", linewidths=.5)
sns.heatmap(df_filtered_log2.head(100), cmap='viridis', annot=False, fmt=".2f", linewidths=.5)

plt.title('Gene Expression Heatmap')
plt.xlabel('Time Points')
plt.ylabel('Genes')
plt.xticks(rotation=45)

# Set gene names as y-axis labels
#plt.yticks(ticks=range(len(gene_names)), labels=gene_names)

plt.tight_layout()
plt.show()


In [ ]:
# Load DataFrame
data = pd.read_csv('output.txt', sep='\t', index_col=0)

df = pd.DataFrame(data)

# Calculate coefficient of variation (CV)
df_cv = df.drop(columns='Gene').apply(lambda x: (x.std() / x.mean()) * 100, axis=1)
df_cv = pd.DataFrame({'Gene': df['Gene'], 'CV': df_cv})

# Sort genes by CV in descending order
df_cv_sorted = df_cv.sort_values(by='CV', ascending=False)

# Define the top n% of varying genes to display
top_n_percent = 10  # Change this value as desired

# Calculate the number of genes to display
num_genes_to_display = int(len(df_cv_sorted) * (top_n_percent / 100))
print(num_genes_to_display)

# Filter the top n% of varying genes
df_top_n_percent = df_cv_sorted.head(num_genes_to_display).copy()

# Filter the original DataFrame to include only the top n% of varying genes
# df_filtered = df[df['Gene'].isin(df_top_n_percent['Gene'])]

# Filter the original DataFrame to include only the top n% of varying genes
df_filtered = df[df['Gene'].isin(df_top_n_percent['Gene'])].copy()

# Remove the 'Gene' column before setting the index
gene_names = df_filtered['Gene']  # Store gene names for later use
df_filtered.drop(columns=['Gene'], inplace=True)

# Generate a clustered heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_filtered, cmap='viridis', annot=False, fmt=".2f", linewidths=.5)
plt.title('Top 50% Varying Genes Heatmap')
plt.xlabel('Time Points')
plt.ylabel('Genes')
plt.xticks(rotation=45)

# Set gene names as y-axis labels
#plt.yticks(ticks=range(len(gene_names)), labels=gene_names)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load DataFrame
data = pd.read_csv('output.txt', sep='\t', index_col=0)

df = pd.DataFrame(data)

# Replace NaN values with zeros
df.fillna(0, inplace=True)
print(df.head(10))

# Store the original index
original_index = df.index

# Remove the 'Gene' column for clustering
gene_names = df['Gene']  # Store gene names for later use
df.drop(columns=['Gene'], inplace=True)

# Define filtering criteria
min_expression = 5.0  # Minimum expression level
min_timepoints_above_threshold = 3  #

# Drop non-varying genes (standard deviation = 0)
df_filtered = df[(df > min_expression).sum(axis=1) >= min_timepoints_above_threshold]
df_filtered = df_filtered[df_filtered.std(axis=1) > 2]

# Insert the 'Gene' column back with original index
df_filtered.insert(0, 'Gene', gene_names)

# Reset the index to original
df_filtered.index = original_index
# Calculate coefficient of variation (CV)
#df_cv = df.drop(columns='Gene').apply(lambda x: (x.std() / x.mean()) * 100, axis=1)
#df_cv = pd.DataFrame({'Gene': df.index, 'CV': df_cv})

# Calculate coefficient of variation (CV) using NumPy functions
means = df_filtered.drop(columns='Gene').mean(axis=1)
stds = df_filtered.drop(columns='Gene').std(axis=1)

# Create a DataFrame for CV with 'Gene' column
df_cv = pd.DataFrame({'Gene': df['Gene'], 'CV': (stds / means) * 100})

#df_cv = (stds / means) * 100
print(df_cv.head(10))

# Reset the index so that 'Gene' becomes a regular column
df_cv.reset_index(drop=True, inplace=True)

# Sort genes by CV in descending order
df_cv_sorted = df_cv.sort_values(by='CV', ascending=False)
print(df_cv_sorted.head(10))

# Define the top n% of varying genes to display
top_n_percent = 1  # Change this value as desired

# Calculate the number of genes to display
num_genes_to_display = int(len(df_cv_sorted) * (top_n_percent / 100))
print(num_genes_to_display)

# Filter the top n% of varying genes
df_top_n_percent = df_cv_sorted.head(num_genes_to_display).copy()
print(df_top_n_percent.tail(10))
print(df_top_n_percent.tail(10)['Gene'])

# Filter the original DataFrame to include only the top n% of varying genes
#df_filtered = df[df.index.isin(df_top_n_percent['Gene'])].copy()
#df_filtered = df[df['Gene'].isin(df_top_n_percent['Gene'])].copy()
gene_names_to_include = df_top_n_percent['Gene'].tolist()
print(gene_names_to_include)
df_filtered = df[df['Gene'].isin(gene_names_to_include)].copy()
print(df_filtered.head(10))

# Remove the 'Gene' column before setting the index
gene_names = df_filtered.index  # Store gene names for later use
df_filtered.drop(columns=['Gene'], inplace=True)

# Generate a clustered heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_filtered, cmap='viridis', annot=False, fmt=".2f", linewidths=.5)
plt.title(f'Top {top_n_percent}% Varying Genes Heatmap')
plt.xlabel('Time Points')
plt.ylabel('Genes')
plt.xticks(rotation=45)

# Set gene names as y-axis labels
#plt.yticks(ticks=range(len(gene_names)), labels=gene_names)

plt.tight_layout()
plt.show()
